In [1]:
%load_ext autoreload

In [ ]:
from opxrd.wrapper import OpXRD

# ------------------------------------------------------------

smoltest_dirpath = '/home/daniel/aimat/data/opXRD/test_smol'
bigtest_dirpath = '/home/daniel/aimat/data/opXRD/test'
test_databases = OpXRD.load_project_list(root_dirpath=bigtest_dirpath)

In [ ]:
from opxrd.analysis import DatabaseAnalyser

%autoreload 2

analyser = DatabaseAnalyser(databases=test_databases, output_dirpath='/tmp/opxrd_analysis')
analyser.plot_effective_components()
analyser.plot_effective_components(use_fractions=False)

In [ ]:
analyser.plot_fourier_reference(b1=60, c1=1, b2=80, c2=1)

In [ ]:
#analyser.plot_fourier()

In [ ]:
analyser.plot_in_single(limit_patterns=10)

In [ ]:
analyser.plot_in_single(limit_patterns=50)

In [ ]:
analyser.plot_in_single(limit_patterns=100)

In [ ]:
analyser.plot_in_single(limit_patterns=1000)

In [ ]:
%autore
analyser.plot_effective_components()

In [ ]:
analyser.plot_histogram()

In [ ]:
analyser.show_label_fractions()
analyser.print_total_counts()